# Setting everything up

In [1]:
#import the file from the web
import requests
from bs4 import BeautifulSoup
import pandas as pd
from unicodedata import normalize
import re
from datetime import datetime
from pandas import read_csv 

# I can give a number or use None to remove maximum ceiling & display all columns
pd.options.display.max_columns = None

# # I want to be able to see the entire narrative, so remove the maximum width for each column
pd.options.display.max_colwidth = None

pd.options.display.float_format = '${:,.2f}'.format

import string

In [2]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (8, 8)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [3]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
%%R

# My commonly used R imports

require('tidyverse')

R[write to console]: Loading required package: tidyverse



── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.1 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.3.0      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 1.0.0 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [5]:
#To Download PDFs
from urllib.parse import urljoin


In [6]:
#To import camelot and PDF-related items
import camelot
import ghostscript
import sys

In [7]:
%%R

library(jpeg)
library(wordcloud)
library(RColorBrewer)
library(wordcloud2)
library(tm)


R[write to console]: Loading required package: RColorBrewer

R[write to console]: Loading required package: NLP

R[write to console]: 
Attaching package: ‘NLP’


R[write to console]: The following object is masked from ‘package:ggplot2’:

    annotate




# For Spotify specifically

### Source:
I got a lot of support for this section from looking at [LUIZ FERNANDO TOLEDO's analysis of Iron Maiden songs](
https://github.com/luizftoledo/luizftoledo.github.io/blob/main/iron-maiden-happiness/docs/data_analysis_maiden.ipynb)

In [8]:
#when posting online use the below so that you hide personal details
# CLIENT_ID = 'INSERT_YOUR_ID'
# CLIENT_SECRET = 'INSERT_YOUR_SECRET'




In [9]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [10]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [11]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# base URL of all Spotify API playlists
BASE_URL_PLAYLISTS = BASE_URL+'playlists/'

# Track ID from the URI
playlist_id = '4lhWxULgawAPOEgAkClW0b'

# actual GET request with proper header
r = requests.get(BASE_URL_PLAYLISTS + playlist_id, headers=headers)


In [12]:
r = r.json()
# r
# r


In [13]:
# WE NEED TO MAKE THREE REQUESTS FOR THE API AS ITS LIMIT IS 100 AND THERE ARE 215 TRACKS THERE.
# WE USE "OFFSET" FEATURE TO DO A FIRST REQUEST FROM PLAYLIST 0 TO 100 AND GET THE FIRST 100 TRACKS, 
# THEN OFFSET 100 TO START FROM WHERE THE FIRST REQUEST ENDED.

playlist_id = '4lhWxULgawAPOEgAkClW0b'

# pull all playlist tracks 
r = requests.get(BASE_URL_PLAYLISTS + playlist_id, 
                 headers=headers, 
                 params={'include_groups': 'track', 'limit': 100, 'offset':0})
d = r.json()

# d['tracks']['items'][0]['track']['name']

# for diction in d['tracks']['items'][0]['track']:
#     print (diction)
#     print ("AAAAHHHHHH")

# ['external_urls']

In [15]:
r2 = requests.get(BASE_URL_PLAYLISTS + playlist_id, 
                 headers=headers, 
                 params={'include_groups': 'track', 'limit': 100, 'offset':0})
d2 = r2.json()
# d.update(d2)

# d2


# for track_num in range(0,100):
#     print(d2['tracks']['items'][track_num]['track']['name'])


In [16]:
r3 = requests.get(BASE_URL_PLAYLISTS + playlist_id, 
                 headers=headers, 
                 params={'include_groups': 'track', 'limit': 15, 'offset':200})
d3 = r3.json()
# d3

d3['tracks']['items'][0]['track']['name']


'Afghanistan [Milli Surood (National Anthem), "This land is Afghanistan…"] (2006 onwards)'

In [17]:
#Merging results into one dataset
# d.update(d2)

# d.update(d3)

# d



# def Merge(dict_1, dict_2):
#     result = dict_1 | dict_2
#     return result

# dict_4 = Merge(d, d2)
# dict_5 = Merge(dict_4, d3)
# dict_5

In [21]:
d3['tracks']['items'][0]['track']

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/50MREZ0LJcSvrtKuyTV8Su'},
    'href': 'https://api.spotify.com/v1/artists/50MREZ0LJcSvrtKuyTV8Su',
    'id': '50MREZ0LJcSvrtKuyTV8Su',
    'name': 'Peter Breiner',
    'type': 'artist',
    'uri': 'spotify:artist:50MREZ0LJcSvrtKuyTV8Su'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/428GNN7qZnTsMaK3SfPo6D'},
    'href': 'https://api.spotify.com/v1/artists/428GNN7qZnTsMaK3SfPo6D',
    'id': '428GNN7qZnTsMaK3SfPo6D',
    'name': 'Slovak Radio Symphony Orchestra',
    'type': 'artist',
    'uri': 'spotify:artist:428GNN7qZnTsMaK3SfPo6D'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/2Eo4VORMdwb15W8WZh3vl6'},
    'href': 'https://api.spotify.com/v1/artists/2Eo4VORMdwb15W8WZh3vl6',
    'id': '2Eo4VORMdwb15W8WZh3vl6',
    'name': 'Slovak State Philharmonic Orchestra, Kosice',
    'type': 'artist',
    'uri': 'spotify:artist:2Eo4VORMdwb15W8WZh

# This is hectic

I need to get the 215 songs through the python library and then to get the track ID in order to proceed with an analysis of valence